In [36]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [37]:
df = pd.read_csv("all_years_combined.csv",encoding='latin1')
df = df.rename(columns={"customers_out_sum": "customers_out"})

df['FIPS'] = df['STATE_FIPS'].astype(str).str.zfill(2) + df['CZ_FIPS'].astype(str).str.zfill(3)
df['FIPS'] = df['FIPS'].astype(int)
df['year'] = df['BEGIN_YEARMONTH'].astype(str).str[:4].astype(int)

df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
df['BEGIN_DATE'] = pd.to_datetime(df['BEGIN_DATE_TIME']).dt.date
df['END_DATE'] = pd.to_datetime(df['END_DATE_TIME']).dt.date

C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\1757311529.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['BEGIN_DATE'] = pd.to_datetime(df['BEGIN_DATE_TIME']).dt.date
C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\1757311529.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['END_DATE'] = pd.to_datetime(df['END_DATE_TIME']).dt.date


In [ ]:
season_map = {
    'DEC': 'Winter', 'JAN': 'Winter', 'FEB': 'Winter',
    'MAR': 'Spring', 'APR': 'Spring', 'MAY': 'Spring',
    'JUN': 'Summer', 'JUL': 'Summer', 'AUG': 'Summer',
    'SEP': 'Fall', 'OCT': 'Fall', 'NOV': 'Fall'
}
df['season'] = df['MONTH_NAME'].map(season_map)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END,customers_out,run_start_time_mean,interval_count,year,BEGIN_DATE,END_DATE,event_duration,avg_event_duration_by_type,season
270372,202207,10,600,202207,10,600,172631,1047980,VIRGINIA,51,...,2022-07-10 06:00:00,0.0,NaN,0,2022,2022-07-10,2022-07-10,0,0.458579,NaN
183070,202002,6,2227,202002,6,2227,144993,870834,FLORIDA,12,...,2020-02-06 22:27:00,0.0,NaN,0,2020,2020-02-06,2020-02-06,0,0.001507,NaN
291611,202304,15,1505,202304,15,1505,179259,1091928,MISSOURI,29,...,2023-04-15 15:05:00,0.0,NaN,0,2023,2023-04-15,2023-04-15,0,0.001979,NaN
43828,201604,10,2201,201604,10,2201,103899,621475,OKLAHOMA,40,...,2016-04-10 22:01:00,0.0,NaN,0,2016,2016-04-10,2016-04-10,0,0.001979,NaN
157758,201906,18,1905,201906,18,1905,137539,826224,TEXAS,48,...,2019-06-18 19:05:00,0.0,NaN,0,2019,2019-06-18,2019-06-18,0,0.001507,NaN


In [40]:
outage_freq = df[df['customers_out'] > 0].groupby('STATE').size()
df['region_outage_freq'] = df['STATE'].map(outage_freq)
df['region_outage_freq'].fillna(0, inplace=True)

C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\3160111230.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['region_outage_freq'].fillna(0, inplace=True)


In [41]:
state_centers = df.groupby('STATE')[['BEGIN_LAT', 'BEGIN_LON']].mean()
df = df.merge(state_centers, on='STATE', suffixes=('', '_CENTER'))

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = phi2 - phi1
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

df['dist_to_state_center'] = haversine(
    df['BEGIN_LAT'], df['BEGIN_LON'],
    df['BEGIN_LAT_CENTER'], df['BEGIN_LON_CENTER']
)

In [42]:
monthly_outage_risk = df.groupby(['STATE', 'MONTH_NAME'])['customers_out'].mean().to_dict()
df['monthly_outage_risk_index'] = df.apply(
    lambda row: monthly_outage_risk.get((row['STATE'], row['MONTH_NAME']), 0), axis=1)

event_freq = df['EVENT_TYPE'].value_counts().to_dict()
df['event_type_freq'] = df['EVENT_TYPE'].map(event_freq)

df['BEGIN_HOUR'] = df['BEGIN_TIME'].astype(str).str.zfill(4).str[:2].astype(int)
df['event_hour_norm'] = df['BEGIN_HOUR'] / 24.0

min_year = df['YEAR'].min()
df['year_trend'] = df['YEAR'] - min_year

df['BEGIN_DATE'] = pd.to_datetime(df['BEGIN_DATE_TIME'])
df['is_weekday'] = df['BEGIN_DATE'].dt.weekday < 5
df['is_weekday'] = df['is_weekday'].astype(int)

wfo_freq = df['WFO'].value_counts().to_dict()
df['wfo_influence'] = df['WFO'].map(wfo_freq).fillna(0)

df['lat_grid'] = df['BEGIN_LAT'].round()
df['lon_grid'] = df['BEGIN_LON'].round()
grid_density = df.groupby(['lat_grid', 'lon_grid']).size().to_dict()
df['grid_density'] = df.apply(lambda row: grid_density.get((row['lat_grid'], row['lon_grid']), 0), axis=1)

df['month_sin'] = np.sin(2 * np.pi * df['BEGIN_DATE'].dt.month / 12)

load_proxy = df.groupby('STATE')['customers_out'].mean().to_dict()
df['grid_load_proxy'] = df['STATE'].map(load_proxy)

C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\4251618459.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['BEGIN_DATE'] = pd.to_datetime(df['BEGIN_DATE_TIME'])


In [44]:
# state_feature_cols = ['STATE'] + [col for col in df.columns if col not in test_df.columns and col != 'STATE']
# state_features = df[state_feature_cols].drop_duplicates('STATE')

# # Step 2: Merge onto test by STATE
# test_df = df.merge(state_features, on='STATE', how='left')
# df.head()
new_cols = [col for col in df.columns if col not in test_df.columns and col != 'STATE']

# Step 2: Keep only state + those new columns
state_features = df[['STATE'] + new_cols].drop_duplicates(subset='STATE')

# Step 3: Merge them into test
test_df = test_df.merge(state_features, on='STATE', how='left')

JORDAN PART

In [46]:
features_test = test_df[['year', 'customers_out', 'BEGIN_DATE_TIME',
    'BEGIN_LAT', 'BEGIN_LON', 'grid_density', 'grid_load_proxy', 'month_sin', 'is_weekday',
    'EVENT_TYPE', 'avg_event_duration_by_type', 'region_outage_freq', 'FIPS', 'wfo_influence']]
features_test.columns = [col.replace('_y', '') if col.endswith('_y') else col for col in features_test.columns]

features_train = df[['year', 'customers_out', 'BEGIN_DATE_TIME',
    'BEGIN_LAT', 'BEGIN_LON', 'grid_density', 'grid_load_proxy', 'month_sin', 'is_weekday',
    'EVENT_TYPE', 'avg_event_duration_by_type', 'region_outage_freq', 'FIPS', 'wfo_influence']]

features_test.head()

,year,customers_out,BEGIN_DATE_TIME,BEGIN_LAT,BEGIN_LON,grid_density,grid_load_proxy,month_sin,is_weekday,EVENT_TYPE,avg_event_duration_by_type,region_outage_freq,FIPS,wfo_influence
0,2020,0.0,20-MAR-20 13:24:00,40.96,-79.19,1818,1024.005365,0.5,0,Thunderstorm Wind,0.001507,2353.0,42065,6580
1,2018,0.0,14-MAY-18 18:45:00,37.94,-76.94,560,5353.025719,-0.5,0,Thunderstorm Wind,0.458579,3107.0,51057,4016
2,2017,35.0,23-JUL-17 10:45:00,39.87,-75.59,1818,1024.005365,0.5,0,Flash Flood,0.001507,2353.0,42045,6580
3,2019,0.0,13-JUN-19 16:12:00,38.65,-77.39,560,5353.025719,-0.5,0,Thunderstorm Wind,0.458579,3107.0,51153,4016
4,2020,0.0,23-JUL-20 17:02:00,43.09,-71.33,484,10253.937037,-1.0,0,Thunderstorm Wind,0.001507,358.0,33015,1797


In [47]:
def time_features(features):
    features['BEGIN_DATE_TIME'] = pd.to_datetime(features['BEGIN_DATE_TIME'])

    #extract features
    features['begin_month'] = features['BEGIN_DATE_TIME'].dt.month
    features['begin_weekday'] = features['BEGIN_DATE_TIME'].dt.dayofweek  # Monday=0

    event_mapping = {event: idx for idx, event in enumerate(sorted(features['EVENT_TYPE'].unique()))}

    def map_event_to_num(event):
        return event_mapping.get(event, -1)

    features['event_type_num'] = features['EVENT_TYPE'].apply(map_event_to_num)

    features = features.drop(columns=['BEGIN_DATE_TIME', 'EVENT_TYPE'])
    print(features.columns.tolist())
    features = features.dropna() #only one row contains NaNs
    return features

features_test = time_features(features_test)
features_train = time_features(features_train)

C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\4155798234.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  features['BEGIN_DATE_TIME'] = pd.to_datetime(features['BEGIN_DATE_TIME'])
C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\4155798234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['BEGIN_DATE_TIME'] = pd.to_datetime(features['BEGIN_DATE_TIME'])
C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\4155798234.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

['year', 'customers_out', 'BEGIN_LAT', 'BEGIN_LON', 'grid_density', 'grid_load_proxy', 'month_sin', 'is_weekday', 'avg_event_duration_by_type', 'region_outage_freq', 'FIPS', 'wfo_influence', 'begin_month', 'begin_weekday', 'event_type_num']
['year', 'customers_out', 'BEGIN_LAT', 'BEGIN_LON', 'grid_density', 'grid_load_proxy', 'month_sin', 'is_weekday', 'avg_event_duration_by_type', 'region_outage_freq', 'FIPS', 'wfo_influence', 'begin_month', 'begin_weekday', 'event_type_num']


C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\4155798234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['BEGIN_DATE_TIME'] = pd.to_datetime(features['BEGIN_DATE_TIME'])
C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\4155798234.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['begin_month'] = features['BEGIN_DATE_TIME'].dt.month
C:\Users\jdsto\AppData\Local\Temp\ipykernel_6940\4155798234.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [51]:
features_test.head()

,year,customers_out,BEGIN_LAT,BEGIN_LON,grid_density,grid_load_proxy,month_sin,is_weekday,avg_event_duration_by_type,region_outage_freq,FIPS,wfo_influence,begin_month,begin_weekday,event_type_num
0,2020,0.0,40.96,-79.19,1818,1024.005365,0.5,0,0.001507,2353.0,42065,6580,3,4,11
1,2018,0.0,37.94,-76.94,560,5353.025719,-0.5,0,0.458579,3107.0,51057,4016,5,0,11
2,2017,35.0,39.87,-75.59,1818,1024.005365,0.5,0,0.001507,2353.0,42045,6580,7,6,3
3,2019,0.0,38.65,-77.39,560,5353.025719,-0.5,0,0.458579,3107.0,51153,4016,6,3,11
4,2020,0.0,43.09,-71.33,484,10253.937037,-1.0,0,0.001507,358.0,33015,1797,7,3,11


In [52]:
features_test.to_csv("TestfeaturesXGBoost.csv", index=False)
features_train.to_csv("TrainFeaturesXGBoost.csv", index=False)